<a href="https://colab.research.google.com/github/donghwi2022/ds-section4-project/blob/main/section4_project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사용 모델
- BERT 모델을 한국어 기반으로 제작한 KoBERT

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, notebook

In [4]:
from google.colab import files

uploaded = files.upload()

Saving dataset.txt to dataset.txt


In [5]:
print("사용한 데이터셋 : https://github.com/2runo/Curse-detection-data")

사용한 데이터셋 : https://github.com/2runo/Curse-detection-data


In [6]:
df = pd.read_csv("dataset.txt", sep="|",
                 header=None, names=['text', 'label'], # column이 없기 때문에 좌측 내용으로 column 생성
                 skiprows=456) # 456번 라인은 중간에 |가 들어가 있어 오류를 유발하므로 해당 라인은 제거
df_sample = df[:1000]
df_sample

,text,label
0,쓰레기통 리뷰는 처음봐서ㅠ,0
1,프리시즌이랑 시즌초반때는 폼이 안좋이서 재계약도 힘들지않을까했는데 중반부터 다시 살...,0
2,찌그러져있어 넌 ㅋㅋ,0
3,빅버드 나올때 사실 내가 삭~ 도망가서 숨었음.,0
4,걍 부산앞바다에 빠져서 죽어라 왜 사냐?? 어휴,1
...,...,...
995,성평등...그는 평등하지 않았다,0
996,은평구 달동네 진짜 무서움.. 밤에다니면 지릴듯,0
997,쪽국은 아직 시작도 안한셈..아마 제대로 검사시작하면 진정한 좀비국될듯...,0
998,옛날 일베하고 지금 일베하고 비교하자면지금 일베글들이 조회수는 훨씬 더 잘나오는 듯...,0


In [7]:
# 정보 확인
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [8]:
# 최고 문장의 길이 확인
text_length = df_sample['text'].apply(len)
print('최고 문장의 길이 :', np.max(text_length))

최고 문장의 길이 : 470


In [9]:
# 학습을 위해 array 형태로 변경
dataset = df_sample.values
dataset

array([['쓰레기통 리뷰는 처음봐서ㅠ', 0],
       ['프리시즌이랑 시즌초반때는 폼이 안좋이서 재계약도 힘들지않을까했는데 중반부터 다시 살아남ㅋㅋ클래스가 있긴있는듯',
        0],
       ['찌그러져있어 넌 ㅋㅋ', 0],
       ...,
       ['쪽국은 아직 시작도 안한셈..아마 제대로 검사시작하면 진정한 좀비국될듯...', 0],
       ['옛날 일베하고 지금 일베하고 비교하자면지금 일베글들이 조회수는 훨씬 더 잘나오는 듯페이지 5 이상만 가봐도 기본 7~8만에좀 자극적인 제목이면 10만 가뿐히 넘음근데 옛날 만베 글들은 조회수 봐라지금보다 못함눈팅족이 많아진듯',
        0],
       ['갑자기 항공방제 비행기가 나타난 느낌?', 0]], dtype=object)

In [10]:
# 학습, 테스트 데이터셋 분리
train, test = train_test_split(dataset, test_size=0.15, random_state=42)
print("학습 데이터의 수:", len(train))
print("테스트 데이터의 수:", len(test))

학습 데이터의 수: 850
테스트 데이터의 수: 150


In [11]:
# 사전 학습된 bert 모델 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# 토크나이저 선언
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
# 학습할 때 사용할 데이터셋 클래스 선언
class BERTDataset(Dataset):
    def __init__(self, dataset, text_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[text_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
# 파라미터 세팅
max_len = 480 # 위에서 구한 최고 문장의 길이가 470이기 때문에 살짝 높은 480으로 설정, 세팅 가능 최대 숫자가 512로 보임
batch_size = 4
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [19]:
# Dataset 선언 
data_train = BERTDataset(train, 0, 1, tokenizer, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tokenizer, max_len, True, False)

# Dataloader 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

In [20]:
# 모델 생성
class BERTClassifier(torch.nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = torch.nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = torch.nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [21]:
# 모델 선언
model = BERTClassifier(bertmodel,  dr_rate=0.5)

# 옵티마이저, 손실함수, 스케줄러 선언
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss() # 이 손실함수는 다중분류인데 이진분류인 BCELoss 함수를 사용하면 손실계산시에 오류 발생


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도를 계산하는 함수 선언
def calc_accuracy(x,y):
    max_vals, max_indices = torch.max(x, 1)
    train_acc = (max_indices == y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
# 학습 진행
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        out = model(token_ids.long(), valid_length, segment_ids.long())
        loss = loss_fn(out, label.long())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label.long())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        out = model(token_ids.long(), valid_length, segment_ids.long())
        test_acc += calc_accuracy(out, label.long())
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/213 [00:00<?, ?it/s]

epoch 1 train acc 0.5880281690140845


  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 test acc 0.7368421052631579


  0%|          | 0/213 [00:00<?, ?it/s]

...

epoch 2 test acc 0.7697368421052632


  0%|          | 0/213 [00:00<?, ?it/s]

epoch 3 train acc 0.7946009389671361


  0%|          | 0/38 [00:00<?, ?it/s]

epoch 3 test acc 0.7763157894736842


In [24]:
# 신뢰도를 계산하는 함수 작성
def reliability(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

# 테스트 모델 작성
def testModel(model, seq):
    cate = ["욕설X","욕설O"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tokenizer, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]), [tokenized[1]], torch.tensor(tokenized[2]))
    idx = result.argmax().cpu().item()
    print("욕설 여부 :", cate[idx])
    print("신뢰도 :", "{:.2f}%".format(reliability(result,idx)))

In [29]:
# 문장 입력받아서 욕설여부 판단
while True :
    sentence = input("문장을 입력해주세요(종료는 '0'을 입력) : ")
    if sentence == '0' :
        break
    testModel(model, sentence)
    print("\n")

문장을 입력해주세요(종료는 '0'을 입력) : 안녕하세요
욕설 여부 : 욕설X
신뢰도 : 93.93%


문장을 입력해주세요(종료는 '0'을 입력) : 야이 미친새끼야 뭐해
욕설 여부 : 욕설O
신뢰도 : 86.22%


문장을 입력해주세요(종료는 '0'을 입력) : 아니 뭐하는 새22222222끼지
욕설 여부 : 욕설O
신뢰도 : 86.71%


문장을 입력해주세요(종료는 '0'을 입력) : ㄹㅇ 똑같은 업무지시를 해도 사람 진짜 존나 빡치게 만드는 재주가 있음
욕설 여부 : 욕설O
신뢰도 : 85.45%


문장을 입력해주세요(종료는 '0'을 입력) : 존나 예쁘네
욕설 여부 : 욕설X
신뢰도 : 80.19%


문장을 입력해주세요(종료는 '0'을 입력) : 대가리 총 맞았냐
욕설 여부 : 욕설X
신뢰도 : 93.84%


문장을 입력해주세요(종료는 '0'을 입력) : 뭐하는데 ㅅㅂ
욕설 여부 : 욕설X
신뢰도 : 93.89%


문장을 입력해주세요(종료는 '0'을 입력) : 0
